In [6]:
# carregar dataset rudementar
# ==============================================================
# CARREGAR DATASET RUDIMENTAR — Coleta de POIs do OSM
# Angola (Pontos Turísticos com coordenadas conhecidas)
# ==============================================================

import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path
import time

# === Diretório de saída ===
DATA_DIR = Path("../data/geoespacial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# === Locais turísticos (com coordenadas conhecidas) ===
locais_turisticos = [
    {"nome": "Quedas de Calandula", "lat": -9.075025, "lon": 16.001131, "provincia": "Malanje"},
    {"nome": "Miradouro da Lua", "lat": -9.221147, "lon": 13.090001, "provincia": "Luanda"},
    {"nome": "Museu Kulumbimbi", "lat": -6.264389, "lon": 14.245581, "provincia": "Zaire"},
    {"nome": "Reserva Parcial do Namibe", "lat": -15.766760, "lon": 12.399914, "provincia": "Namibe"},
    {"nome": "Fortaleza de São Miguel", "lat": -8.808343416048587, "lon": 13.223444621289627, "provincia": "Luanda"},
    {"nome": "Ilha do Mussulo", "lat": -8.964635, "lon":13.050462, "provincia": "Luanda"},
    {"nome": "Cânion do Rio Kalandula", "lat": -9.074959, "lon": 16.001378, "provincia": "Malanje"},
    {"nome": "Parque Nacional da Quiçama", "lat": -9.750007, "lon": 13.583333, "provincia": "Luanda"},
    {"nome": "Cristo Rei", "lat": -14.940039, "lon": 13.511860, "provincia": "Huambo"},
    {"nome": "Serra da Leba", "lat": -15.0788, "lon": 13.2397, "provincia": "Huíla"},
    {"nome": "Fenda da Tundavala", "lat": -14.817465, "lon": 13.381539, "provincia": "Huíla"},
    {"nome": "Baía Azul", "lat": -12.627978, "lon": 13.234185, "provincia": "Benguela"},
    {"nome": "Quedas do Rio Chiumbe", "lat": -11.021380, "lon": 20.203180, "provincia": "Moxico"},
    {"nome": "Parque Nacional de Cameia", "lat": -11.883338, "lon": 21.666681, "provincia": "Moxico"},
    {"nome": "Grutas do Nzenzo", "lat": -7.520192, "lon":14.565103, "provincia": "Uíge"}
]

BUFFER_RADIUS = 5000  # 5 km
dfs = []

# === Tags OSM — Turismo + Infraestrutura ===
TAGS = {
    "tourism": True,
    "amenity": True,
    "leisure": True,
    "natural": True,
    "landuse": True,
    "highway": True,
    "waterway": True,
    "power": True,
    "man_made": True,
    "railway": True,
    "building": True
}


def coletar_pois(lat, lon, tags, buffer_m):
    """Retorna os POIs dentro do raio (buffer_m) ao redor das coordenadas."""
    try:
        gdf_ponto = gpd.GeoDataFrame([["ponto", Point(lon, lat)]],
                                     columns=["nome", "geometry"],
                                     crs="EPSG:4326")
        area_busca = gdf_ponto.to_crs(3857).buffer(buffer_m).to_crs(4326).iloc[0]

        pois = ox.features_from_polygon(area_busca, tags).reset_index()
        if pois.empty:
            return None

        # Garantir colunas
        colunas_necessarias = [
            "osm_id", "name", "tourism", "amenity", "leisure",
            "natural", "landuse", "highway", "waterway",
            "power", "man_made", "railway", "building", "geometry"
        ]
        for c in colunas_necessarias:
            if c not in pois.columns:
                pois[c] = None
        return pois[colunas_necessarias]
    except Exception as e:
        print(f" Erro na coleta: {e}")
        return None


# === Loop pelos locais ===
for item in locais_turisticos:
    nome = item["nome"]
    lat = item["lat"]
    lon = item["lon"]
    prov = item["provincia"]

    print(f"\n Coletando POIs para {nome} ({prov})...")

    pois = coletar_pois(lat, lon, TAGS, BUFFER_RADIUS)

    if pois is not None:
        # Adicionar metadados
        pois["ponto_turistico"] = nome
        pois["provincia"] = prov
        pois["fonte_dados"] = "OpenStreetMap"
        pois["latitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).y
        pois["longitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).x

        # Salvar individualmente
        output_path = DATA_DIR / f"pois_{nome.replace(' ', '_').lower()}.geojson"
        pois.to_file(output_path, driver="GeoJSON")
        print(f"    Salvo: {output_path}")

        dfs.append(pois)
    else:
        print(f" Nenhum POI encontrado em {nome}.")

    time.sleep(1.5)  # respeitar limite de requisições

# === Combinar tudo ===
if dfs:
    pois_final = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True), crs="EPSG:4326")
    output_final = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
    pois_final.to_file(output_final, driver="GeoJSON")
    print(f"\n Arquivo combinado salvo em: {output_final}")
else:
    print("\n Nenhum POI foi coletado.")



 Coletando POIs para Quedas de Calandula (Malanje)...
    Salvo: ..\data\geoespacial\pois_quedas_de_calandula.geojson

 Coletando POIs para Miradouro da Lua (Luanda)...
    Salvo: ..\data\geoespacial\pois_miradouro_da_lua.geojson

 Coletando POIs para Museu Kulumbimbi (Zaire)...
    Salvo: ..\data\geoespacial\pois_museu_kulumbimbi.geojson

 Coletando POIs para Reserva Parcial do Namibe (Namibe)...
    Salvo: ..\data\geoespacial\pois_reserva_parcial_do_namibe.geojson

 Coletando POIs para Fortaleza de São Miguel (Luanda)...
    Salvo: ..\data\geoespacial\pois_fortaleza_de_são_miguel.geojson

 Coletando POIs para Ilha do Mussulo (Luanda)...
    Salvo: ..\data\geoespacial\pois_ilha_do_mussulo.geojson

 Coletando POIs para Cânion do Rio Kalandula (Malanje)...
    Salvo: ..\data\geoespacial\pois_cânion_do_rio_kalandula.geojson

 Coletando POIs para Parque Nacional da Quiçama (Luanda)...
 Erro na coleta: No matching features. Check query location, tags, and log.
 Nenhum POI encontrado em Pa

In [1]:
# Normalizar 

# ==============================================================
# NORMALIZAÇÃO DOS DADOS GEOFÍSICOS E DE INFRAESTRUTURA
# ==============================================================

import geopandas as gpd
from pathlib import Path

DATA_DIR = Path("../data/geoespacial")
input_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
output_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson"

# ===  Carregar dados brutos ===
pois = gpd.read_file(input_path)

# ===  Limpeza básica ===
if "osm_id" in pois.columns:
    pois = pois.drop_duplicates(subset=["osm_id"], keep="first")
else:
    pois = pois.drop_duplicates(subset=["name", "geometry"], keep="first")

pois = pois[pois.is_valid]
pois = pois.dropna(subset=["geometry"])

# ===  Garantir colunas críticas ===
for col in ["name", "tourism", "amenity", "leisure", "natural", "landuse", 
            "highway", "power", "latitude", "longitude"]:
    if col not in pois.columns:
        pois[col] = None

# ===  Correção e normalização ===
pois["name"] = pois["name"].fillna("Desconhecido").astype(str)

map_tourism = {
    "guest_house": "accommodation",
    "hotel": "accommodation",
    "motel": "accommodation",
    "museum": "culture",
    "attraction": "attraction"
}
map_amenity = {
    "restaurant": "food_service",
    "fast_food": "food_service",
    "hospital": "health",
    "school": "education"
}

pois["tourism"] = pois["tourism"].replace(map_tourism)
pois["amenity"] = pois["amenity"].replace(map_amenity)

# ===  Variáveis derivadas ===
pois["is_touristic"] = pois["tourism"].notna().astype(int)
pois["is_infrastructure"] = pois[["amenity", "highway", "power"]].notna().any(axis=1).astype(int)

# ===  Salvar resultado ===
pois.to_file(output_path, driver="GeoJSON")
print(f" Dados normalizados salvos em: {output_path}")




 Dados normalizados salvos em: ..\data\geoespacial\pontos_turisticos_angola_normalizado.geojson


In [ ]:
# ==============================================================
# NORMALIZAÇÃO DOS DADOS DE MOBILIDADE E DE INFRAESTRUTURA
# ==============================================================

import pandas as pd
import geopandas as gpd
from pathlib import Path

# === Diretório e arquivo ===
DATA_DIR = Path("../data/climatic-environmental")
DATA_DIR.mkdir(parents=True, exist_ok=True)
input_path = DATA_DIR / "mobilidade_infra.csv"

# === Carregar ===
df = pd.read_csv(input_path)

# === Normalizar nomes de colunas ===
df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)

# === Preenchimento básico ===
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
for c in num_cols:
    df[c] = df[c].fillna(df[c].median())

cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    df[c] = df[c].fillna("desconhecido")

# === Limites plausíveis ===
if "velocidade_media_kmh" in df.columns:
    df["velocidade_media_kmh"] = df["velocidade_media_kmh"].clip(lower=0, upper=200)

if "capacidade" in df.columns:
    df["capacidade"] = df["capacidade"].clip(lower=0)

if "fluxo_diario" in df.columns:
    df["fluxo_diario"] = df["fluxo_diario"].clip(lower=0)

# === Datas ===
for date_col in ["date", "data", "timestamp"]:
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# === Normalizar categorias de infraestrutura ===
infra_col_candidates = [c for c in df.columns if c in ("tipo", "tipo_infra", "infra_type", "category")]
if infra_col_candidates:
    infra_col = infra_col_candidates[0]
    df[infra_col] = df[infra_col].astype(str).str.lower()
    df["is_road"] = df[infra_col].str.contains(r"road|highway|estrada", na=False).astype(int)
    df["is_rail"] = df[infra_col].str.contains(r"rail|ferrovia", na=False).astype(int)
    df["is_port"] = df[infra_col].str.contains(r"port|porto", na=False).astype(int)
    df["is_airport"] = df[infra_col].str.contains(r"airport|aeroporto", na=False).astype(int)

# === Coordenadas -> GeoDataFrame ===
lat_candidates = [c for c in df.columns if c in ("latitude", "lat", "y")]
lon_candidates = [c for c in df.columns if c in ("longitude", "lon", "lng", "x")]

output_csv = DATA_DIR / "mobilidade_infra_normalizado.csv"
output_geo = DATA_DIR / "mobilidade_infra_normalizado.geojson"

if lat_candidates and lon_candidates:
    lat_col = lat_candidates[0]
    lon_col = lon_candidates[0]
    df = df.dropna(subset=[lat_col, lon_col])
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col].astype(float), df[lat_col].astype(float)),
        crs="EPSG:4326"
    )
    gdf.to_file(output_geo, driver="GeoJSON")
    df.to_csv(output_csv, index=False)
    print(f"Dados de mobilidade normalizados salvos em:\n   - {output_csv}\n   - {output_geo}")
else:
    df.to_csv(output_csv, index=False)
    print(f"Dados de mobilidade normalizados salvos em: {output_csv}")



✅ Dados de mobilidade normalizados salvos em: ..\data\climatic-environmental\mobilidade_infra_normalizado.csv


In [17]:
import pandas as pd
from pathlib import Path

# ==============================================================
# NORMALIZAÇÃO DOS DADOS CLIMATICOS-AMBIENTAIS

# ==============================================================
# === Diretório e arquivos ===
DATA_DIR = Path("../data/climatic-environmental")
DATA_DIR.mkdir(parents=True, exist_ok=True)

input_clima = DATA_DIR / "exemplo.csv"
df_clima = pd.read_csv(input_clima)

# === Normalização dos dados ===
# Temperatura média anual (°C)
df_clima['temp_med_anual'] = df_clima['temp_med_anual'].clip(20, 35)

# Precipitação anual (mm)
df_clima['precipitacao_anual'] = df_clima['precipitacao_anual'].clip(0, 2000)

# Índices de vegetação e água
for col in ['NDVI', 'EVI', 'NDWI']:
    df_clima[col] = df_clima[col].clip(-1, 1)

# Códigos LULC (Land Use Land Cover)
df_clima['LULC_codigo'] = df_clima['LULC_codigo'].astype('category')

# Altitude (m)
df_clima['altitude'] = df_clima['altitude'].clip(0, 2500)

# === Salvar dados normalizados ===
output_path = DATA_DIR / "clima_lulc_normalizado.csv"
df_clima.to_csv(output_path, index=False)
print(f" Dados climáticos-ambientais normalizados salvos em: {output_path}")

 Dados climáticos-ambientais normalizados salvos em: ..\data\climatic-environmental\clima_lulc_normalizado.csv


In [7]:
import pandas as pd
from pathlib import Path

# ==============================================================
# NORMALIZAÇÃO DOS DADOS ECONÔMICOS-SOCIAIS
# ==============================================================
# === Diretório e arquivos ===
DATA_DIR = Path("../data/economicsocial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

input_eco = DATA_DIR / "dados_socioeconomicos.csv"
df_eco = pd.read_csv(input_eco)

# === Normalização dos dados ===
# PIB per capita (USD)
df_eco['pib_per_capita'] = df_eco['pib_per_capita'].clip(500, 5000)

# População
df_eco['populacao'] = df_eco['populacao'].clip(1000, 1000000)

# Índice de desenvolvimento humano
df_eco['idh'] = df_eco['idh'].clip(0, 1)

# Densidade populacional (hab/km²)
df_eco['densidade_pop'] = df_eco['densidade_pop'].clip(0, 1000)

# Taxa de urbanização (%)
df_eco['taxa_urbanizacao'] = df_eco['taxa_urbanizacao'].clip(0, 100)

# === Salvar dados normalizados ===
output_path = DATA_DIR / "economicsocial_normalizado.csv"
df_eco.to_csv(output_path, index=False)
print(f" Dados econômicos-sociais normalizados salvos em: {output_path}")

 Dados econômicos-sociais normalizados salvos em: ..\data\economicsocial\economicsocial_normalizado.csv


In [2]:
# Gerar variáveis espaciais 
# ============================================================== 
# GERAR VARIÁVEIS ESPACIAIS
# ==============================================================

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from pathlib import Path
import numpy as np

DATA_DIR = Path("../data/geoespacial")
input_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson"
output_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson"

# === Carregar dados normalizados ===
pois = gpd.read_file(input_path)

# === Converter para projeção métrica (metros) para cálculos espaciais ===
pois = pois.to_crs(epsg=3857)

# === Exemplo 1: Calcular centroid de cada ponto turístico (se houver polígonos) ===
# (No caso de pontos simples, é só o próprio ponto)
pois['centroid_x'] = pois.geometry.centroid.x
pois['centroid_y'] = pois.geometry.centroid.y

# === Exemplo 2: Distância até o ponto turístico mais próximo ===
# Criar uma função para calcular a menor distância entre cada ponto
def nearest_distance(point, others):
    # Retorna a menor distância (em metros) até outro ponto
    distances = others.geometry.distance(point)
    distances = distances[distances > 0]  # ignora distância zero (o próprio ponto)
    if len(distances) > 0:
        return distances.min()
    else:
        return np.nan

pois['dist_to_nearest_poi'] = pois.geometry.apply(lambda p: nearest_distance(p, pois))

# === Exemplo 3: Buffer / contagem de POIs próximos (densidade) ===
BUFFER_RADIUS = 1000  # metros
pois['pois_within_1km'] = pois.geometry.apply(lambda g: pois[pois.geometry.within(g.buffer(BUFFER_RADIUS))].shape[0] - 1) 
# -1 para excluir o próprio ponto

# === Exemplo 4: Flag de proximidade a categorias específicas ===
# Por exemplo, se existe hospital ou escola em 1km
def has_category_nearby(point, category_col, category_value, radius=1000):
    buffer = point.buffer(radius)
    subset = pois[(pois[category_col] == category_value) & (pois.geometry.within(buffer))]
    return int(len(subset) > 0)

pois['hospital_nearby'] = pois.geometry.apply(lambda g: has_category_nearby(g, 'amenity', 'hospital'))
pois['school_nearby'] = pois.geometry.apply(lambda g: has_category_nearby(g, 'amenity', 'school'))

# === Salvar dataset com variáveis espaciais ===
pois = pois.to_crs(epsg=4326)  # voltar para lat/lon
pois.to_file(output_path, driver="GeoJSON")
print(f" Variáveis espaciais geradas e salvas em: {output_path}")


 Variáveis espaciais geradas e salvas em: ..\data\geoespacial\pontos_turisticos_angola_raw.geojson


In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import re

# ============================================================
# 🔹 FUNÇÕES AUXILIARES
# ============================================================

def normalizar(texto):
    """Remove acentos, espaços extras e converte em minúsculas."""
    if pd.isna(texto):
        return ""
    texto = str(texto).strip().lower()
    texto = re.sub(r"[áàãâä]", "a", texto)
    texto = re.sub(r"[éèêë]", "e", texto)
    texto = re.sub(r"[íìîï]", "i", texto)
    texto = re.sub(r"[óòõôö]", "o", texto)
    texto = re.sub(r"[úùûü]", "u", texto)
    texto = re.sub(r"ç", "c", texto)
    texto = re.sub(r"[^a-z0-9 ]", "", texto)
    return texto


def carregar_csv(caminho):
    """Lê CSV corrigindo encoding e normalizando nomes de colunas."""
    df = pd.read_csv(caminho, encoding="utf-8", on_bad_lines="skip")
    df.columns = [normalizar(c) for c in df.columns]
    return df


def adicionar_chave(df, colunas_possiveis):
    """Adiciona coluna 'key' com nome normalizado."""
    for col in df.columns:
        if any(col.startswith(c) for c in colunas_possiveis):
            df["key"] = df[col].astype(str).apply(normalizar)
            return df
    raise ValueError(f"❌ Nenhuma coluna de nome encontrada. Colunas disponíveis: {list(df.columns)}")


# ============================================================
# 🔹 CAMINHOS DOS DADOS
# ============================================================

BASE_DIR = Path("../data")
PATH_GEO_NORM = BASE_DIR / "geoespacial/pontos_turisticos_angola_normalizado.geojson"
PATH_GEO_RAW = BASE_DIR / "geoespacial/pontos_turisticos_angola_raw.geojson"
PATH_CLIMA = BASE_DIR / "climatic-environmental/clima_lulc_normalizado.csv"
PATH_MOB = BASE_DIR / "climatic-environmental/mobilidade_infra_normalizado.csv"
PATH_SOCIO = BASE_DIR / "economicsocial/economicsocial_normalizado.csv"

# ============================================================
# 🔹 CARREGAR CAMADAS
# ============================================================

print("🔹 Carregando camadas de dados...")

gdf_norm = gpd.read_file(PATH_GEO_NORM)
gdf_raw = gpd.read_file(PATH_GEO_RAW)
df_clima = carregar_csv(PATH_CLIMA)
df_mob = carregar_csv(PATH_MOB)
df_socio = carregar_csv(PATH_SOCIO)

# ============================================================
# 🔹 ADICIONAR CHAVES NORMALIZADAS
# ============================================================

print("🔹 Normalizando nomes e criando chaves...")

gdf_norm = adicionar_chave(gdf_norm, ["ponto_turistico", "name", "nome"])
gdf_raw = adicionar_chave(gdf_raw, ["ponto_turistico", "name", "nome"])
df_clima = adicionar_chave(df_clima, ["nome_ponto_turistico", "nome"])
df_mob = adicionar_chave(df_mob, ["nome"])
df_socio = adicionar_chave(df_socio, ["nome"])

# ============================================================
# 🔹 JUNÇÃO SEMÂNTICA — CLIMA + SOCIO + MOBILIDADE
# ============================================================

print("🔹 Realizando junção semântica (clima + socioeconômico + mobilidade)...")

df_semantico = df_clima.merge(df_socio, on="key", how="outer", suffixes=("_clima", "_socio"))
df_semantico = df_semantico.merge(df_mob, on="key", how="outer", suffixes=("", "_mob"))

# ============================================================
# 🔹 JUNÇÃO ESPACIAL — COM GEOESPACIAL NORMALIZADO E RAW
# ============================================================

print("🔹 Realizando junção espacial com dados geográficos...")

# Converter para GeoDataFrame com base em coordenadas (lat/lon)
if "longitude" in gdf_norm.columns and "latitude" in gdf_norm.columns:
    gdf_norm["geometry"] = gdf_norm.apply(lambda x: Point(x["longitude"], x["latitude"]), axis=1)
gdf_norm = gpd.GeoDataFrame(gdf_norm, geometry="geometry", crs="EPSG:4326")

# Merge final com base na chave
gdf_final = gdf_norm.merge(df_semantico, on="key", how="left")
gdf_final = gdf_final.merge(gdf_raw.drop(columns=["geometry"]), on="key", how="left", suffixes=("", "_raw"))

# ============================================================
# 🔹 EXPORTAR RESULTADOS
# ============================================================

OUTPUT_DIR = BASE_DIR / "integrado"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

csv_out = OUTPUT_DIR / "dataset_integrado.csv"
geojson_out = OUTPUT_DIR / "dataset_integrado.geojson"

print(f" Salvando arquivos em: {OUTPUT_DIR}")
gdf_final.to_csv(csv_out, index=False, encoding="utf-8")
gdf_final.to_file(geojson_out, driver="GeoJSON")

print(" Integração concluída com sucesso!")
print(f" Arquivos gerados:\n - {csv_out.name}\n - {geojson_out.name}")


🔹 Carregando camadas de dados...
🔹 Normalizando nomes e criando chaves...
🔹 Realizando junção semântica (clima + socioeconômico + mobilidade)...
🔹 Realizando junção espacial com dados geográficos...
 Salvando arquivos em: ..\data\integrado
 Integração concluída com sucesso!
 Arquivos gerados:
 - dataset_integrado.csv
 - dataset_integrado.geojson


In [ ]:
import pandas as pd
from pathlib import Path

# =====================================================
#  Gerar model_input.csv consolidado e pronto p/ ML
# =====================================================

# Caminhos dos arquivos
DATA_CLIMA = Path("../data/climatic-environmental/clima_lulc_normalizado.csv")
DATA_MOB = Path("../data/climatic-environmental/mobilidade_infra_normalizado.csv")
DATA_SOCIO = Path("../data/economicsocial/economicsocial_normalizado.csv")

# === 1. Carregar dados ===
df_clima = pd.read_csv(DATA_CLIMA)
df_mob = pd.read_csv(DATA_MOB)
df_socio = pd.read_csv(DATA_SOCIO)

# === 2. Normalizar nomes e chaves ===
def normalize_name(n):
    return (
        str(n)
        .strip()
        .lower()
        .replace("ã", "a")
        .replace("ç", "c")
        .replace("í", "i")
        .replace("é", "e")
        .replace("ú", "u")
        .replace("ó", "o")
    )

for df in [df_clima, df_mob, df_socio]:
    df.columns = df.columns.str.strip().str.lower()
    if "nome_ponto_turistico" in df.columns:
        df["key"] = df["nome_ponto_turistico"].apply(normalize_name)
    elif "nome" in df.columns:
        df["key"] = df["nome"].apply(normalize_name)

# === 3. Merge dos datasets ===
df_merge = (
    df_clima.merge(df_socio, on="key", suffixes=("_clima", "_socio"))
    .merge(df_mob, on="key", suffixes=("", "_mob"))
)

# === 4. Selecionar colunas finais ===
colunas_finais = [
    "nome_ponto_turistico_clima",
    "provincia_clima",
    "lat_clima",
    "lon_clima",
    "temp_med_anual",
    "precipitacao_anual",
    "ndvi",
    "evi",
    "ndwi",
    "altitude",
    "populacao",
    "densidade_pop",
    "pib_per_capita",
    "idh",
    "taxa_urbanizacao",
    "emprego_turismo",
    "distancia_estrada_principal_km",
    "distancia_cidade_km",
]

# Garantir que as colunas existam
colunas_existentes = [c for c in colunas_finais if c in df_merge.columns]

# === 5. Criar dataframe final ===
df_final = df_merge[colunas_existentes].copy()

# === 6. Salvar CSV final ===
output_path = Path("../data/model_input.csv")
df_final.to_csv(output_path, index=False)

print(f" Model Input salvo em: {output_path}")
print(df_final.head())

✅ Model Input salvo em: ..\data\model_input.csv
  nome_ponto_turistico_clima provincia_clima  lat_clima  lon_clima  \
0        Quedas de Calandula         Malanje  -9.075025  16.001131   
1           Miradouro da Lua          Luanda  -9.221147  13.090001   
2           Museu Kulumbimbi           Zaire  -6.264389  14.245581   
3  Reserva Parcial do Namibe          Namibe -15.766760  12.399914   
4    Fortaleza de São Miguel          Luanda  -8.808343  13.223445   

   temp_med_anual  precipitacao_anual  ndvi   evi  ndwi  altitude  populacao  \
0            28.8               218.9  0.80  0.16  0.02     287.0      95533   
1            24.8               304.1  0.77  0.68  0.08    1148.0      52641   
2            27.8               343.0  0.20  0.46  0.08     683.0      54008   
3            24.3               286.7  0.15  0.60  0.04     362.0     166079   
4            28.1               158.6  0.62  0.47  0.16    1192.0     192436   

   densidade_pop  pib_per_capita    idh  taxa_urba